### Objective
In this notebook, I am attempting to do tranfer learning with Keras, Tensorflow. This is based on the tutorial at [keras.org](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html).

I modified the script located in a [gist](https://gist.githubusercontent.com/fchollet/f35fbc80e066a49d65f1688a7e99f069/raw/04f05ef9d573acb503476d07123097ba99181f3c/classifier_from_little_data_script_2.py). 

#### Load the required modules and set path to the data

In [83]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.applications import vgg16
from keras.utils.data_utils import get_file
from keras import optimizers
from os.path import join, split
from os import getcwd
import h5py
import numpy as np
from keras import backend as K
from keras.utils.np_utils import convert_kernel
import tensorflow as tf


# path to the model weights file
base = join(split(split(getcwd())[0])[0],'data_local','dogs_cats')
weights_path = join(split(base)[0],'vgg16_weights.h5')
top_model_weights_path = join(base, 'bottleneck_fc_model.h5')

# dimensions of the imates
img_width , img_height = 150, 150

train_data_dir = join(base,'train')
validation_data_dir = join(base,'validation')
nb_train_samples = 2000
nb_validation_sampeles = 800
nb_epoch = 10




In [8]:
def save_bottleneck_feature(train_data_dir, validation_data_dir, save_directory,
                           img_width =150, img_height=150,nb_train_samples=62*32,
                            nb_validation_samples=800):
    """Predicts with the convolutional layers of VGG16 and saves the output
    Args: 
        train_data_dir(str): path to training data
        validation__data_dir(str): path to validation data
        save_directory(str): path to save the files
    Returns None
    """

    # load the vgg16 covolutional layers
    model = vgg16.VGG16(include_top=False,input_shape=( img_width, img_height,3))
    
    # set up a data generator for the training data
    datagen=ImageDataGenerator(1./255)
    generator = datagen.flow_from_directory(
            train_data_dir,
            target_size=(img_width, img_height),
            batch_size=32,
            class_mode=None,
            shuffle=False)
    
    # predict on the training data
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples)
    file_name = join(save_directory, 'bottleneck_features_train.npy')
    np.save(open(file_name, 'w'), bottleneck_features_train)
    
    # set up a generator for the validation_data
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size = (img_width, img_height),
        batch_size=32,
        class_mode = None,
        shuffle=False)
    
    # predict on the validation data
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples)
    file_name = join(save_directory, 'bottleneck_features_validation.npy')
    np.save(open(file_name, 'w'), bottleneck_features_validation)
    
    return None


In [10]:
save_bottleneck_feature(train_data_dir=train_data_dir, validation_data_dir=validation_data_dir,
                       save_directory=base, nb_train_samples=62*32, nb_validation_samples=800)

Found 2002 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [34]:
def train_top_model(save_directory,top_model_weights_path,
                    nb_train_samples=32*62, nb_validation_samples=800,
                   nb_epochs=50 ):
    """ Trains the top model weights"""
    file_name = join(save_directory, 'bottleneck_features_train.npy')
    train_data = np.load(open(file_name))
    train_labels = np.array([0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))
    
    
    file_name = join(save_directory, 'bottleneck_features_validation.npy')
    validation_data = np.load(open(file_name))
    validation_labels = np.array([0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    # set up the 
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              nb_epoch=nb_epoch, batch_size=32,
              validation_data=(validation_data, validation_labels), 
             verbose = 1)
    
    model.save_weights(top_model_weights_path)
    return model

    
    

In [35]:
top_model = train_top_model(save_directory = base,top_model_weights_path=top_model_weights_path)

Train on 1984 samples, validate on 800 samples
Epoch 1/50
1984/1984 [==============================] - 1s - loss: 3.0749 - acc: 0.7984 - val_loss: 3.7050 - val_acc: 0.7612
Epoch 2/50
1984/1984 [==============================] - 1s - loss: 2.0987 - acc: 0.8599 - val_loss: 1.1869 - val_acc: 0.9187
Epoch 3/50
1984/1984 [==============================] - 1s - loss: 1.5605 - acc: 0.8962 - val_loss: 1.1901 - val_acc: 0.9200
Epoch 4/50
1984/1984 [==============================] - 1s - loss: 1.2378 - acc: 0.9204 - val_loss: 1.0665 - val_acc: 0.9287
Epoch 5/50
1984/1984 [==============================] - 1s - loss: 1.2872 - acc: 0.9153 - val_loss: 0.9512 - val_acc: 0.9350
Epoch 6/50
1984/1984 [==============================] - 1s - loss: 1.2671 - acc: 0.9183 - val_loss: 1.0268 - val_acc: 0.9325
Epoch 7/50
1984/1984 [==============================] - 1s - loss: 1.1348 - acc: 0.9264 - val_loss: 1.5012 - val_acc: 0.9025
Epoch 8/50
1984/1984 [==============================] - 2s - loss: 1.0541 - ac

In [44]:
def get_weights_path():
    """gets the local path to the weights"""
    TF_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
    return get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                        TF_WEIGHTS_PATH_NO_TOP,
                                        cache_subdir='models')
    
    

In [84]:
def setup_alternate_VGG16_model(top_model=None, img_width = 150, img_height = 150,
                               freeze_layers = 25,loss=None, optimizer=None, metrics=None):
    """Sets up an alternate vgg16 model with diffent top"""

    # build the VGG16 network
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))

    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    # Load the weights
    model.load_weights(get_weights_path())
    
    if top_model is None:
        # build a classifier model to put on top of the convolutional model
        top_model = Sequential()
        top_model.add(Flatten(input_shape=model.output_shape[1:]))
        top_model.add(Dense(256, activation='relu'))
        top_model.add(Dropout(0.5))
        top_model.add(Dense(1, activation='sigmoid'))
        
        top_model_weights_path = base = join(split(split(getcwd())[0])[0],'data_local','dogs_cats',
                                       'bottleneck_fc_model.h5')
        
        top_model.load_weights(top_model_weights_path)
        
    
    model.add(top_model)
    
    # freeze the first 25 layers
    # set the first 25 layers (up to the last conv block)
    for layer in model.layers[:freeze_layers]:
        layer.trainable = False
    
    # compile the model to be optimzied
    if loss is None: 
        loss = 'binary_crossentropy'
    if optimizer is None: 
        optimizer = optimizers.SGD(lr=1e-4, momentum=0.9)
    if metrics is None:
        metrics = ['accuracy']

    model.compile(loss =loss, 
                  optimizer = optimizer,
                  metrics = metrics)
                  
    
    
    return model 

In [90]:
model = setup_alternate_VGG16_model(top_model=None, img_width = 150, img_height = 150)

#### Set up the training of the model 

In [92]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

Found 2002 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [94]:
# fine-tune the model
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=800)

Epoch 1/50
1984/2000 [============================>.] - ETA: 4s - loss: 1.2838 - acc: 0.7908 

/Users/crivera5/.virtual_envs/Kaggle/lib/python2.7/site-packages/keras/engine/training.py:1527: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


2002/2000 [==============================] - 778s - loss: 1.2812 - acc: 0.7917 - val_loss: 0.3249 - val_acc: 0.9075
Epoch 2/50
2002/2000 [==============================] - 763s - loss: 0.4976 - acc: 0.8616 - val_loss: 0.2861 - val_acc: 0.9175
Epoch 3/50
2002/2000 [==============================] - 745s - loss: 0.3825 - acc: 0.8856 - val_loss: 0.2765 - val_acc: 0.9175
Epoch 4/50
2002/2000 [==============================] - 741s - loss: 0.2906 - acc: 0.9001 - val_loss: 0.2620 - val_acc: 0.9175
Epoch 5/50
2002/2000 [==============================] - 738s - loss: 0.2344 - acc: 0.9201 - val_loss: 0.2255 - val_acc: 0.9225
Epoch 6/50
2002/2000 [==============================] - 731s - loss: 0.2001 - acc: 0.9206 - val_loss: 0.2141 - val_acc: 0.9237
Epoch 7/50
2002/2000 [==============================] - 733s - loss: 0.1718 - acc: 0.9356 - val_loss: 0.2127 - val_acc: 0.9213
Epoch 8/50
2002/2000 [==============================] - 733s - loss: 0.1474 - acc: 0.9436 - val_loss: 0.2064 - val_acc: 0.

### Conclusion 
This general method worked for doing the training on the dog cat example. We can use it to base our initial training of the other models. 

The one issue is that we will want to use the a different top model. 

In [95]:
type(top_model)

keras.models.Sequential